# Feature engineering

## Obtain the data

In [2]:
import os
import pandas as pd

In [3]:
# Earliest possible date is 2017-06-17
from_date = '2017-07-01'
until_date = '2017-07-31'

local_data_folder = 'data/raw' # do not end in /
output_folder = 'data/processed' # do not end in /

download_script = './download_data.sh'

dates = list(pd.date_range(from_date, until_date, freq='D').strftime('%Y-%m-%d'))

! mkdir -p {local_data_folder}

# We found it was more reliable to generate a bash script and run it, rather than
# run the commands in a python for-loop

with open(download_script, 'w') as f:
    f.write("#!/bin/bash\n")
    f.write("\nset -euo pipefail\n")
    f.write("\n# This script was generated to download data for multiple days\n")
    for date in dates:
        success_file =  os.path.join(local_data_folder, date, 'success')

        f.write("""
if [ ! -f {success_file} ]; then

    echo "Getting PDS dataset for date {date}"        
    mkdir -p {local_data_folder}/{date}
    aws s3 sync s3://deutsche-boerse-xetra-pds/{date} {local_data_folder}/{date} --no-sign-request
    touch {success_file}            
else
    echo "PDS dataset for date {date} already exists"
fi\n""".format(success_file=success_file, date=date, local_data_folder=local_data_folder))

        
! chmod +x {download_script}     
! head -n 15 {download_script} 

#!/bin/bash

set -euo pipefail

# This script was generated to download data for multiple days

if [ ! -f data/raw/2017-07-01/success ]; then

    echo "Getting PDS dataset for date 2017-07-01"        
    mkdir -p data/raw/2017-07-01
    aws s3 sync s3://deutsche-boerse-xetra-pds/2017-07-01 data/raw/2017-07-01 --no-sign-request
    touch data/raw/2017-07-01/success            
else
    echo "PDS dataset for date 2017-07-01 already exists"
fi


In [4]:
# execute the download script to retrieve the data
!  {download_script}

Getting PDS dataset for date 2017-07-01
download: s3://deutsche-boerse-xetra-pds/2017-07-01/2017-07-01_BINS_XETR02.csv to data/raw/2017-07-01/2017-07-01_BINS_XETR02.csv
download: s3://deutsche-boerse-xetra-pds/2017-07-01/2017-07-01_BINS_XETR01.csv to data/raw/2017-07-01/2017-07-01_BINS_XETR01.csv
download: s3://deutsche-boerse-xetra-pds/2017-07-01/2017-07-01_BINS_XETR07.csv to data/raw/2017-07-01/2017-07-01_BINS_XETR07.csv
download: s3://deutsche-boerse-xetra-pds/2017-07-01/2017-07-01_BINS_XETR00.csv to data/raw/2017-07-01/2017-07-01_BINS_XETR00.csv
download: s3://deutsche-boerse-xetra-pds/2017-07-01/2017-07-01_BINS_XETR03.csv to data/raw/2017-07-01/2017-07-01_BINS_XETR03.csv
download: s3://deutsche-boerse-xetra-pds/2017-07-01/2017-07-01_BINS_XETR06.csv to data/raw/2017-07-01/2017-07-01_BINS_XETR06.csv
download: s3://deutsche-boerse-xetra-pds/2017-07-01/2017-07-01_BINS_XETR10.csv to data/raw/2017-07-01/2017-07-01_BINS_XETR10.csv
download: s3://deutsche-boerse-xetra-pds/2017-07-01/2017-

## Cleanse the data

In [5]:
import pandas as pd
import numpy as np
import glob, os
from datetime import datetime
import statsmodels.api as sm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
mpl.rcParams['figure.figsize'] = (15, 10) # use bigger graphs

In [8]:
def load_csv_dirs(data_dirs):
    files = []
    for data_dir in data_dirs:
        files.extend(glob.glob(os.path.join(data_dir, '*.csv')))
    return pd.concat(map(pd.read_csv, files))

data_dir = local_data_folder + '/'
data_subdirs = map(lambda date: data_dir + date, dates)
unprocessed_df = load_csv_dirs(data_subdirs)
unprocessed_df.head(2)

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT00BUWOG001,BWO,BUWOG AG,Common stock,EUR,2504156,2017-07-01,08:29,25.090,25.090,25.090,25.090,0,1
1,AT000KAPSCH9,BZ6,KAPSCH TRAFFICCOM AG,Common stock,EUR,2504157,2017-07-01,08:29,46.715,46.715,46.715,46.715,0,1


In [9]:
unprocessed_df.shape

(1342577, 14)

In [10]:
unprocessed_df.info ()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1342577 entries, 0 to 6152
Data columns (total 14 columns):
ISIN              1342577 non-null object
Mnemonic          1342577 non-null object
SecurityDesc      1342577 non-null object
SecurityType      1342577 non-null object
Currency          1342577 non-null object
SecurityID        1342577 non-null object
Date              1342577 non-null object
Time              1342577 non-null object
StartPrice        1342577 non-null float64
MaxPrice          1342577 non-null float64
MinPrice          1342577 non-null float64
EndPrice          1342577 non-null float64
TradedVolume      1342577 non-null object
NumberOfTrades    1342577 non-null object
dtypes: float64(4), object(10)
memory usage: 153.6+ MB


In [11]:
unprocessed_df.count()

ISIN              1342577
Mnemonic          1342577
SecurityDesc      1342577
SecurityType      1342577
Currency          1342577
SecurityID        1342577
Date              1342577
Time              1342577
StartPrice        1342577
MaxPrice          1342577
MinPrice          1342577
EndPrice          1342577
TradedVolume      1342577
NumberOfTrades    1342577
dtype: int64

In [12]:
unprocessed_df.Mnemonic.value_counts()

DAI     10610
SAP     10557
BAS     10547
BMW     10532
DBK     10519
EOAN    10505
TKA     10271
FRE     10209
CBK     10163
LHA     10161
ALV     10138
DTE     10051
VOW3    10038
DPW      9997
HEI      9958
SIE      9906
IFX      9905
PSM      9826
BAYN     9762
RWE      9616
CON      9555
ADS      9529
VNA      9513
BEI      9509
FME      9493
DB1      9489
MUV2     9107
G1A      9016
KGX      8751
1COV     8614
        ...  
ECBC        1
X0DP        1
5CH3        1
OD72        1
3QQQ        1
WTD8        1
BNQJ        1
18MX        1
LYXQ        1
OI1S        1
EFL         1
00XN        1
X00Y        1
PSWD        1
FUSA        1
PV3A        1
WTIF        1
SXRL        1
4RT7        1
SSN         1
QDVB        1
EM1C        1
X00P        1
X04V        1
540C        1
XBJG        1
ETLV        1
X04G        1
LDB         1
BCFU        1
Name: Mnemonic, Length: 2357, dtype: int64

In [13]:
# we want the dates to be comparable to datetime.strptime()
unprocessed_df["CalcTime"] = pd.to_datetime("1900-01-01 " + unprocessed_df["Time"])
unprocessed_df["CalcDateTime"] = pd.to_datetime(unprocessed_df["Date"] + " " + unprocessed_df["Time"])
unprocessed_df.head()

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcTime,CalcDateTime
0,AT00BUWOG001,BWO,BUWOG AG,Common stock,EUR,2504156,2017-07-01,08:29,25.090,25.090,25.090,25.090,0,1,1900-01-01 08:29:00,2017-07-01 08:29:00
1,AT000KAPSCH9,BZ6,KAPSCH TRAFFICCOM AG,Common stock,EUR,2504157,2017-07-01,08:29,46.715,46.715,46.715,46.715,0,1,1900-01-01 08:29:00,2017-07-01 08:29:00
2,AT0000APOST4,O3P,OESTERREICH. POST AG,Common stock,EUR,2504158,2017-07-01,08:29,38.095,38.095,38.095,38.095,0,1,1900-01-01 08:29:00,2017-07-01 08:29:00
3,AT0000A0E9W5,SANT,S+T AG (Z.REG.MK.Z.)O.N.,Common stock,EUR,2504159,2017-07-01,08:29,13.175,13.175,13.175,13.175,0,1,1900-01-01 08:29:00,2017-07-01 08:29:00
4,AT0000A00XX9,P4N,POLYTEC HLDG AG INH. EO 1,Common stock,EUR,2504160,2017-07-01,08:29,16.590,16.590,16.590,16.590,0,1,1900-01-01 08:29:00,2017-07-01 08:29:00


In [14]:
# Filter common stock
# Filter between trading hours 08:00 and 20:00
# Exclude auctions (those are with TradeVolume == 0)
only_common_stock = unprocessed_df[unprocessed_df.SecurityType == 'Common stock']
time_fmt = "%H:%M"
opening_hours_str = "08:00"
closing_hours_str = "20:00"
opening_hours = datetime.strptime(opening_hours_str, time_fmt)
closing_hours = datetime.strptime(closing_hours_str, time_fmt)

cleaned_common_stock = only_common_stock[(only_common_stock.TradedVolume > 0) & \
                  (only_common_stock.CalcTime >= opening_hours) & \
                  (only_common_stock.CalcTime <= closing_hours)]
cleaned_common_stock.head(2)

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcTime,CalcDateTime
0,CH0012142631,CLRN,"CLARIANT NA SF 3,70",Common stock,EUR,2504222,2017-07-03,09:00,19.54,19.54,19.54,19.54,200,2,1900-01-01 09:00:00,2017-07-03 09:00:00
1,CH0038389992,BBZA,"BB BIOTECH NAM. SF 0,20",Common stock,EUR,2504244,2017-07-03,09:00,53.80,53.80,53.80,53.80,11,1,1900-01-01 09:00:00,2017-07-03 09:00:00


In [15]:
bymnemonic = cleaned_common_stock[['Mnemonic', 'TradedVolume']].groupby(['Mnemonic']).sum()
number_of_stocks = 100
top = bymnemonic.sort_values(['TradedVolume'], ascending=[0]).head(number_of_stocks)
top.head(10)

,TradedVolume
Mnemonic,
DBK,191393378
CBK,162467213
EOAN,122653304
DTE,104769351
LHA,71522999
IFX,56434276
DAI,53841441
SNH,51670309
RWE,45485636


In [16]:
top_k_stocks = list(top.index.values)
cleaned_common_stock = cleaned_common_stock[cleaned_common_stock.Mnemonic.isin(top_k_stocks)]
cleaned_common_stock.head()

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcTime,CalcDateTime
4,DE000A0B65S3,PA8,PAION O.N,Common stock,EUR,2504283,2017-07-03,09:00,3.051,3.051,3.045,3.051,5000,3,1900-01-01 09:00:00,2017-07-03 09:00:00
14,DE000A0WMPJ6,AIXA,AIXTRON SE NA O.N.,Common stock,EUR,2504428,2017-07-03,09:00,6.212,6.212,6.212,6.212,1444,1,1900-01-01 09:00:00,2017-07-03 09:00:00
15,DE000A1EWWW0,ADS,ADIDAS AG NA O.N.,Common stock,EUR,2504471,2017-07-03,09:00,168.850,168.850,168.850,168.850,182,4,1900-01-01 09:00:00,2017-07-03 09:00:00
16,DE000A1PHFF7,BOSS,HUGO BOSS AG NA O.N.,Common stock,EUR,2504512,2017-07-03,09:00,61.420,61.420,61.420,61.420,60,1,1900-01-01 09:00:00,2017-07-03 09:00:00
19,DE000A2AADD2,IGY,INNOGY SE INH. O.N.,Common stock,EUR,2504618,2017-07-03,09:00,34.525,34.525,34.525,34.525,68,1,1900-01-01 09:00:00,2017-07-03 09:00:00


In [17]:
sorted_by_index = cleaned_common_stock.set_index(['Mnemonic', 'CalcDateTime']).sort_index()
sorted_by_index.head()

ISIN       SecurityDesc  SecurityType  \
Mnemonic CalcDateTime                                                         
1COV     2017-07-03 08:00:00  DE0006062144  COVESTRO AG  O.N.  Common stock   
         2017-07-03 08:01:00  DE0006062144  COVESTRO AG  O.N.  Common stock   
         2017-07-03 08:02:00  DE0006062144  COVESTRO AG  O.N.  Common stock   
         2017-07-03 08:04:00  DE0006062144  COVESTRO AG  O.N.  Common stock   
         2017-07-03 08:05:00  DE0006062144  COVESTRO AG  O.N.  Common stock   

                             Currency SecurityID        Date   Time  \
Mnemonic CalcDateTime                                                 
1COV     2017-07-03 08:00:00      EUR    2505008  2017-07-03  08:00   
         2017-07-03 08:01:00      EUR    2505008  2017-07-03  08:01   
         2017-07-03 08:02:00      EUR    2505008  2017-07-03  08:02   
         2017-07-03 08:04:00      EUR    2505008  2017-07-03  08:04   
         2017-07-03 08:05:00      EUR    2505008  2017-07-03  08:05   

                              StartPrice  MaxPrice  MinPrice  EndPrice  \
Mnemonic CalcDateTime                                                    
1COV     2017-07-03 08:00:00       63.34     63.34     63.34     63.34   
         2017-07-03 08:01:00       63.36     63.36     63.36     63.36   
         2017-07-03 08:02:00       63.31     63.31     63.20     63.20   
         2017-07-03 08:04:00       63.22     63.24     63.19     63.19   
         2017-07-03 08:05:00       63.19     63.19     63.17     63.17   

                             TradedVolume NumberOfTrades            CalcTime  
Mnemonic CalcDateTime                                                         
1COV     2017-07-03 08:00:00          103              2 1900-01-01 08:00:00  
         2017-07-03 08:01:00          148              1 1900-01-01 08:01:00  
         2017-07-03 08:02:00         1180             12 1900-01-01 08:02:00  
         2017-07-03 08:04:00          334              4 1900-01-01 08:04:00  
         2017-07-03 08:05:00          533              6 1900-01-01 08:05:00

In [18]:
non_empty_days = sorted(list(cleaned_common_stock['Date'].unique()))
len(non_empty_days), non_empty_days[0:2], '...', non_empty_days[-3:-1]

(21, ['2017-07-03', '2017-07-04'], '...', ['2017-07-27', '2017-07-28'])

In [19]:
print ("Ideal data count for any stock: {}".format (44*12*60))
print ("Observation count per mnemonic:")
cleaned_common_stock.Mnemonic.value_counts()

Ideal data count for any stock: 31680
Observation count per mnemonic:


DAI     9360
SAP     9310
BAS     9298
BMW     9283
DBK     9265
EOAN    9264
TKA     9060
FRE     8989
LHA     8921
CBK     8915
ALV     8900
DTE     8807
VOW3    8796
DPW     8781
HEI     8767
IFX     8670
SIE     8665
PSM     8652
BAYN    8525
RWE     8461
DB1     8373
BEI     8371
CON     8369
VNA     8359
FME     8342
ADS     8307
G1A     7958
MUV2    7930
KGX     7738
1COV    7633
        ... 
TUI1    4389
DRI     4322
ADV     4211
AR4     4119
GYC     3911
GFT     3310
TEG     3295
TINA    3207
SANT    3198
PBB     3164
GAZ     3125
SGL     3012
AOX     2969
MEO     2964
ZIL2    2927
BPE5    2886
ABR     2524
MLP     2340
NOA3    2327
TTI     1934
QSC     1907
WCMK    1517
PNE3    1463
VODI    1405
PA8     1315
ECX     1308
ANO      640
LLD      180
BIR       97
SVAB      94
Name: Mnemonic, Length: 100, dtype: int64

In [20]:
import datetime
def build_index(non_empty_days, from_time, to_time):
    date_ranges = []
    for date in non_empty_days:
        yyyy, mm, dd = date.split('-')
        from_hour, from_min = from_time.split(':')
        to_hour, to_min = to_time.split(':')    
        t1 = datetime.datetime(int(yyyy), int(mm), int(dd), int(from_hour),int(from_min),0)
        t2 = datetime.datetime(int(yyyy), int(mm), int(dd), int(to_hour),int(to_min),0) 
        date_ranges.append(pd.DataFrame({"OrganizedDateTime": pd.date_range(t1, t2, freq='1Min').values}))
    agg = pd.concat(date_ranges, axis=0) 
    agg.index = agg["OrganizedDateTime"]
    return agg
new_datetime_index = build_index(non_empty_days, opening_hours_str, closing_hours_str)["OrganizedDateTime"].values
new_datetime_index

array(['2017-07-03T08:00:00.000000000', '2017-07-03T08:01:00.000000000',
       '2017-07-03T08:02:00.000000000', ...,
       '2017-07-31T19:58:00.000000000', '2017-07-31T19:59:00.000000000',
       '2017-07-31T20:00:00.000000000'], dtype='datetime64[ns]')

In [21]:
def basic_stock_features(input_df, mnemonic, new_time_index):
    stock = sorted_by_index.loc[mnemonic].copy()
    
    stock = stock.reindex(new_time_index)
    
    features = ['MinPrice', 'MaxPrice', 'EndPrice', 'StartPrice']
    for f in features:
        stock[f] = stock[f].fillna(method='ffill')   
    
    features = ['TradedVolume', 'NumberOfTrades']
    for f in features:
        stock[f] = stock[f].fillna(0.0)
        
    stock['HourOfDay'] = stock.index.hour
    stock['MinOfHour'] = stock.index.minute
    stock['MinOfDay'] = stock.index.hour*60 + stock.index.minute

    stock['DayOfWeek'] = stock.index.dayofweek
    stock['DayOfYear'] = stock.index.dayofyear
    stock['MonthOfYear'] = stock.index.month
    stock['WeekOfYear'] = stock.index.weekofyear
    
    stock['Mnemonic'] = mnemonic
    unwanted_features = ['ISIN', 'SecurityDesc', 'SecurityType', 'Currency', 'SecurityID', 'Date', 'Time', 'CalcTime']
    return stock.drop (unwanted_features, axis=1)

In [23]:
s = sorted_by_index.loc['DAI'].copy ()

In [24]:
s.index.dayofyear

Int64Index([184, 184, 184, 184, 184, 184, 184, 184, 184, 184,
            ...
            212, 212, 212, 212, 212, 212, 212, 212, 212, 212],
           dtype='int64', name='CalcDateTime', length=9360)

In [25]:
stocks = []
for stock in top_k_stocks:
    stock = basic_stock_features(sorted_by_index, stock, new_datetime_index)
    stocks.append(stock)
# prepared should contain the numeric features for all top k stocks,
# for all days in the interval, for which there were trades (that means excluding weekends and holidays)
# for all minutes from 08:00 until 20:00
# in minutes without trades the prices from the last available minute are carried forward
# trades are filled with zero for such minutes
# a new column called HasTrade is introduced to denote the presence of trades
prepared = pd.concat(stocks, axis=0)

In [26]:
prepared.Mnemonic = prepared.Mnemonic.astype('category')
prepared.Mnemonic.value_counts()

ZIL2    15141
FME     15141
DEZ     15141
DLG     15141
DPW     15141
DRI     15141
DTE     15141
DWNI    15141
ECX     15141
EOAN    15141
EVK     15141
EVT     15141
FNTN    15141
ZAL     15141
FRE     15141
G1A     15141
GAZ     15141
GBF     15141
GFT     15141
GYC     15141
HDD     15141
HEI     15141
HEN3    15141
IFX     15141
DEQ     15141
DBK     15141
DB1     15141
DAI     15141
ABR     15141
ADS     15141
        ...  
VODI    15141
VOW3    15141
WCMK    15141
WDI     15141
SDF     15141
SAP     15141
LEO     15141
SANT    15141
LHA     15141
LIN     15141
LLD     15141
LXS     15141
MEO     15141
MLP     15141
MRK     15141
MUV2    15141
NDX1    15141
NOA3    15141
O2D     15141
OSR     15141
PA8     15141
PAH3    15141
PBB     15141
PNE3    15141
PSM     15141
QIA     15141
QSC     15141
RKET    15141
RWE     15141
1COV    15141
Name: Mnemonic, Length: 100, dtype: int64

**TODO** Convert timestamp to more meaningful derived features

**TODO** Integrate with plotly for histograms, correlation matrices, etc

In [27]:
sorted_by_index.loc['DAI'].tail ()

,ISIN,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcTime
CalcDateTime,,,,,,,,,,,,,,
2017-07-31 15:25:00,DE0007100000,DAIMLER AG NA O.N.,Common stock,EUR,2505076,2017-07-31,15:25,59.33,59.33,59.29,59.31,10315,32,1900-01-01 15:25:00
2017-07-31 15:26:00,DE0007100000,DAIMLER AG NA O.N.,Common stock,EUR,2505076,2017-07-31,15:26,59.31,59.31,59.28,59.30,15639,30,1900-01-01 15:26:00
2017-07-31 15:27:00,DE0007100000,DAIMLER AG NA O.N.,Common stock,EUR,2505076,2017-07-31,15:27,59.31,59.31,59.30,59.31,7319,22,1900-01-01 15:27:00
2017-07-31 15:28:00,DE0007100000,DAIMLER AG NA O.N.,Common stock,EUR,2505076,2017-07-31,15:28,59.32,59.34,59.31,59.31,14725,91,1900-01-01 15:28:00
2017-07-31 15:29:00,DE0007100000,DAIMLER AG NA O.N.,Common stock,EUR,2505076,2017-07-31,15:29,59.31,59.37,59.31,59.36,20715,43,1900-01-01 15:29:00


In [28]:
prepared.tail()

,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,HourOfDay,MinOfHour,MinOfDay,DayOfWeek,DayOfYear,MonthOfYear,WeekOfYear,Mnemonic
CalcDateTime,,,,,,,,,,,,,,
2017-07-31 19:56:00,36.99,37.01,36.965,36.965,0.0,0.0,19,56,1196,0,212,7,31,SOW
2017-07-31 19:57:00,36.99,37.01,36.965,36.965,0.0,0.0,19,57,1197,0,212,7,31,SOW
2017-07-31 19:58:00,36.99,37.01,36.965,36.965,0.0,0.0,19,58,1198,0,212,7,31,SOW
2017-07-31 19:59:00,36.99,37.01,36.965,36.965,0.0,0.0,19,59,1199,0,212,7,31,SOW
2017-07-31 20:00:00,36.99,37.01,36.965,36.965,0.0,0.0,20,0,1200,0,212,7,31,SOW


In [29]:
prepared.head()

,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,HourOfDay,MinOfHour,MinOfDay,DayOfWeek,DayOfYear,MonthOfYear,WeekOfYear,Mnemonic
CalcDateTime,,,,,,,,,,,,,,
2017-07-03 08:00:00,15.830,15.830,15.820,15.825,6594.0,9.0,8,0,480,0,184,7,27,DBK
2017-07-03 08:01:00,15.835,15.835,15.795,15.800,120179.0,29.0,8,1,481,0,184,7,27,DBK
2017-07-03 08:02:00,15.800,15.805,15.775,15.785,32404.0,29.0,8,2,482,0,184,7,27,DBK
2017-07-03 08:03:00,15.785,15.815,15.785,15.805,34169.0,58.0,8,3,483,0,184,7,27,DBK
2017-07-03 08:04:00,15.805,15.805,15.795,15.805,26589.0,31.0,8,4,484,0,184,7,27,DBK


In [62]:
# We save both in csv in pickle. Generally we'd read from the pickeled format because 
# it preserves the indices, but for cases where pkl cannot be read, we also output a csv format
! mkdir -p {output_folder}
prepared.to_csv(output_folder + '/cooked_v3.csv')

prepared.to_pickle(output_folder + '/cooked_v3.pkl')

In [26]:
!ls -lh {output_folder}

total 385M
-rw-r--r-- 1 825712516 1896053708 149M Dec 10 21:34 cooked_v3.csv
-rw-r--r-- 1 825712516 1896053708 213M Dec 10 21:34 cooked_v3.pkl
